<a href="https://colab.research.google.com/github/kcarroth-dev/KarltonCarrothers_ML/blob/main/ECGR4105_Hw7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score

In [23]:
# Load CIFAR-10 dataset to calculate mean and std
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Load Normalized CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define Convolutional Neural Network
class CNN(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(CNN, self).__init__()

        # Convolution Layer
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

        # Convolutional Layer
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

#ResNet-10 model
class ResNet10(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, n_chans1=64):
        super(ResNet10, self).__init__()
        self.in_channels = n_chans1

        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(n_chans1)
        self.relu = nn.ReLU()
        self.layer1 = self._make_layer(block, n_chans1, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, n_chans1 * 2, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, n_chans1 * 4, num_blocks[2], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(n_chans1 * 4, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Instantiate the ResNet-10 model with ResNet blocks
model = ResNet10(CNN, [4, 3, 3]).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Model Training
num_epochs = 300
total_start_time = time.time()
for epoch in range(num_epochs):
    start_time = time.time()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    end_time = time.time()
    training_time = end_time - start_time

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Training Time: {training_time:.2f} seconds')

total_end_time = time.time()
total_training_time = total_end_time - total_start_time
print(f'Total Training Time: {total_training_time:.2f} seconds')

# Model Evaluation
model.eval()
correct = 0
total = 0
all_predicted = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = correct / total
print(f'Final Test Accuracy: {accuracy * 100:.2f}%')



Epoch 1/300, Loss: 1.8290543197975744, Training Time: 21.17 seconds
Epoch 11/300, Loss: 0.6817077270249272, Training Time: 20.64 seconds
Epoch 21/300, Loss: 0.1791041426341552, Training Time: 20.72 seconds
Epoch 31/300, Loss: 0.028237364527023853, Training Time: 20.82 seconds
Epoch 41/300, Loss: 0.011113108452701288, Training Time: 20.74 seconds
Epoch 51/300, Loss: 0.008055606189414935, Training Time: 20.81 seconds
Epoch 61/300, Loss: 0.005191369487605084, Training Time: 20.91 seconds
Epoch 71/300, Loss: 0.009978081582693497, Training Time: 20.73 seconds
Epoch 81/300, Loss: 0.0028142215434850083, Training Time: 20.68 seconds
Epoch 91/300, Loss: 0.002931999276837875, Training Time: 20.73 seconds
Epoch 101/300, Loss: 0.002171738959928972, Training Time: 20.63 seconds
Epoch 111/300, Loss: 0.003996128597229188, Training Time: 20.79 seconds
Epoch 121/300, Loss: 0.00800091037089797, Training Time: 20.80 seconds
Epoch 131/300, Loss: 0.0012009977246374142, Training Time: 21.04 seconds
Epoch 14

NameError: name 'precision_score' is not defined

In [25]:
# Calculate and print F1 score, precision, and recall
f1 = f1_score(all_labels, all_predicted, average='weighted')
precision = precision_score(all_labels, all_predicted, average='weighted')
recall = recall_score(all_labels, all_predicted, average='weighted')

print(f'Final F1 Score: {f1:.4f}')
print(f'Final Precision: {precision:.4f}')
print(f'Final Recall: {recall:.4f}')

# Calculate and print confusion matrix at the end
cm = confusion_matrix(all_labels, all_predicted)
print('Final Confusion Matrix:')
print(cm)

Final F1 Score: 0.6740
Final Precision: 0.7113
Final Recall: 0.6642
Final Confusion Matrix:
[[654  34  37  21   3 126  10  43  40  32]
 [  9 830   4   9   0  72   4   5   6  61]
 [ 46   4 539  71  37 188  59  40  10   6]
 [  2   1  56 523  22 322  45  21   4   4]
 [ 22   4 106 102 425 144  70 118   4   5]
 [  2   0  28 158  34 735  11  27   1   4]
 [  4   2  37 100  16 105 727   4   1   4]
 [  6   1  18  33  22 202   2 706   0  10]
 [ 68  41  10  22   0 102   4   7 712  34]
 [ 16  71   4  15   1  78   3  12   9 791]]


In [26]:
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score
f1 = f1_score(all_labels, all_predicted, average='weighted')
precision = precision_score(all_labels, all_predicted, average='weighted')
recall = recall_score(all_labels, all_predicted, average='weighted')

print(f'Final F1 Score: {f1:.4f}')
print(f'Final Precision: {precision:.4f}')
print(f'Final Recall: {recall:.4f}')

# Calculate and print confusion matrix
cm = confusion_matrix(all_labels, all_predicted)
print('Final Confusion Matrix:')
print(cm)

Final F1 Score: 0.6740
Final Precision: 0.7113
Final Recall: 0.6642
Final Confusion Matrix:
[[654  34  37  21   3 126  10  43  40  32]
 [  9 830   4   9   0  72   4   5   6  61]
 [ 46   4 539  71  37 188  59  40  10   6]
 [  2   1  56 523  22 322  45  21   4   4]
 [ 22   4 106 102 425 144  70 118   4   5]
 [  2   0  28 158  34 735  11  27   1   4]
 [  4   2  37 100  16 105 727   4   1   4]
 [  6   1  18  33  22 202   2 706   0  10]
 [ 68  41  10  22   0 102   4   7 712  34]
 [ 16  71   4  15   1  78   3  12   9 791]]


In [27]:
# Load CIFAR-10
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformation with calculated mean and std
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Load Normalized CIFAR-10
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define Convolutional Neural Network
class CNN(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(CNN, self).__init__()

        # Convolutional Layer
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

        # Convolutional Layer
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

# Define ResNet-10 model
class ResNet10(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, n_chans1=64):
        super(ResNet10, self).__init__()
        self.in_channels = n_chans1

        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(n_chans1)
        self.relu = nn.ReLU()
        self.layer1 = self._make_layer(block, n_chans1, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, n_chans1 * 2, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, n_chans1 * 4, num_blocks[2], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(n_chans1 * 4, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Instantiate the ResNet-10 model with ResNet blocks
model = ResNet10(CNN, [4, 3, 3]).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=0.001)

# Model Training
num_epochs = 300
total_start_time = time.time()
for epoch in range(num_epochs):
    start_time = time.time()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    end_time = time.time()
    training_time = end_time - start_time

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Training Time: {training_time:.2f} seconds')

total_end_time = time.time()
total_training_time = total_end_time - total_start_time
print(f'Total Training Time: {total_training_time:.2f} seconds')

# Model Evaluation
model.eval()
correct = 0
total = 0
all_predicted = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = correct / total
print(f'Final Test Accuracy: {accuracy * 100:.2f}%')

# Calculate and print F1 score, precision, and recall
f1 = f1_score(all_labels, all_predicted, average='weighted')
precision = precision_score(all_labels, all_predicted, average='weighted')
recall = recall_score(all_labels, all_predicted, average='weighted')

print(f'Final F1 Score: {f1:.4f}')
print(f'Final Precision: {precision:.4f}')
print(f'Final Recall: {recall:.4f}')

# Calculate and print confusion matrix
cm = confusion_matrix(all_labels, all_predicted)
print('Final Confusion Matrix:')
print(cm)

Epoch 1/300, Loss: 1.8370715245566405, Training Time: 21.05 seconds
Epoch 11/300, Loss: 0.6849995207451188, Training Time: 20.81 seconds
Epoch 21/300, Loss: 0.18762281314110207, Training Time: 20.83 seconds
Epoch 31/300, Loss: 0.03068528662238013, Training Time: 20.96 seconds
Epoch 41/300, Loss: 0.01102988062845662, Training Time: 20.87 seconds
Epoch 51/300, Loss: 0.006886657848156265, Training Time: 20.92 seconds
Epoch 61/300, Loss: 0.004378930031595266, Training Time: 20.67 seconds
Epoch 71/300, Loss: 0.004870077243635712, Training Time: 20.86 seconds
Epoch 81/300, Loss: 0.004436143251169292, Training Time: 20.80 seconds
Epoch 91/300, Loss: 0.002417404338328318, Training Time: 20.88 seconds
Epoch 101/300, Loss: 0.002389681308031263, Training Time: 20.87 seconds
Epoch 111/300, Loss: 0.002040942951054622, Training Time: 20.85 seconds
Epoch 121/300, Loss: 0.018811908250069006, Training Time: 20.89 seconds
Epoch 131/300, Loss: 0.002922769788841126, Training Time: 20.74 seconds
Epoch 141/

In [28]:
# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformation with calculated mean and std
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Load Normalized CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define Convolutional Neural Network
class CNN(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, dropout_prob=0.3):
        super(CNN, self).__init__()

        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_prob)

        # Second convolutional layer
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.dropout2 = nn.Dropout(dropout_prob)

        # Shortcut connection
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.Dropout(dropout_prob)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.dropout1(out)
        out = self.bn2(self.conv2(out))
        out = self.dropout2(out)
        out += self.shortcut(x)
        out = self.relu(out)
        return out

# Define ResNet-10 model
class ResNet10(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, n_chans1=64, dropout_prob=0.3):
        super(ResNet10, self).__init__()
        self.in_channels = n_chans1

        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(n_chans1)
        self.relu = nn.ReLU()
        self.layer1 = self._make_layer(block, n_chans1, num_blocks[0], stride=1, dropout_prob=dropout_prob)
        self.layer2 = self._make_layer(block, n_chans1 * 2, num_blocks[1], stride=2, dropout_prob=dropout_prob)
        self.layer3 = self._make_layer(block, n_chans1 * 4, num_blocks[2], stride=2, dropout_prob=dropout_prob)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(n_chans1 * 4, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride, dropout_prob):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride, dropout_prob))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Instantiate the ResNet-10 model with ResNet blocks
model = ResNet10(CNN, [4, 3, 3], dropout_prob=0.3).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Model Training
num_epochs = 300
total_start_time = time.time()
for epoch in range(num_epochs):
    start_time = time.time()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    end_time = time.time()
    training_time = end_time - start_time

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Training Time: {training_time:.2f} seconds')

total_end_time = time.time()
total_training_time = total_end_time - total_start_time
print(f'Total Training Time: {total_training_time:.2f} seconds')

# Model Evaluation
model.eval()
correct = 0
total = 0
all_predicted = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = correct / total
print(f'Final Test Accuracy: {accuracy * 100:.2f}%')

# Calculate and print F1 score, precision, and recall
f1 = f1_score(all_labels, all_predicted, average='weighted')
precision = precision_score(all_labels, all_predicted, average='weighted')
recall = recall_score(all_labels, all_predicted, average='weighted')

print(f'Final F1 Score: {f1:.4f}')
print(f'Final Precision: {precision:.4f}')
print(f'Final Recall: {recall:.4f}')

# Calculate and print confusion matrix
cm = confusion_matrix(all_labels, all_predicted)
print('Final Confusion Matrix:')
print(cm)

Epoch 1/300, Loss: 2.1929081071673147, Training Time: 21.75 seconds
Epoch 11/300, Loss: 1.4337500507569374, Training Time: 21.77 seconds
Epoch 21/300, Loss: 1.1377560472701822, Training Time: 21.82 seconds
Epoch 31/300, Loss: 0.9706312327281289, Training Time: 22.10 seconds
Epoch 41/300, Loss: 0.8682814081154211, Training Time: 21.73 seconds
Epoch 51/300, Loss: 0.7951992017686215, Training Time: 21.65 seconds
Epoch 61/300, Loss: 0.7336129161250561, Training Time: 21.66 seconds
Epoch 71/300, Loss: 0.6776345964054318, Training Time: 21.77 seconds
Epoch 81/300, Loss: 0.6303484962724358, Training Time: 21.79 seconds
Epoch 91/300, Loss: 0.5927398870591922, Training Time: 21.74 seconds
Epoch 101/300, Loss: 0.5545790942047563, Training Time: 21.82 seconds
Epoch 111/300, Loss: 0.524302867024451, Training Time: 21.81 seconds
Epoch 121/300, Loss: 0.4903403527443976, Training Time: 21.47 seconds
Epoch 131/300, Loss: 0.46865252757926124, Training Time: 22.08 seconds
Epoch 141/300, Loss: 0.44231051

In [29]:
# Load CIFAR-10 dataset to calculate mean and std
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformation with calculated mean and std
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Load CIFAR-10 dataset with normalization
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define Convolutional Neural Network
class CNN(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(CNN, self).__init__()

        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

        # Second convolutional layer
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Shortcut connection
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

# Define ResNet-10 model
class ResNet10(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, n_chans1=64):
        super(ResNet10, self).__init__()
        self.in_channels = n_chans1

        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(n_chans1)
        self.relu = nn.ReLU()
        self.layer1 = self._make_layer(block, n_chans1, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, n_chans1 * 2, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, n_chans1 * 4, num_blocks[2], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(n_chans1 * 4, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Instantiate the ResNet-10 model with ResNet blocks and batch normalization
model = ResNet10(CNN, [4, 3, 3]).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Model Training
num_epochs = 300
total_start_time = time.time()
for epoch in range(num_epochs):
    start_time = time.time()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    end_time = time.time()
    training_time = end_time - start_time

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Training Time: {training_time:.2f} seconds')

total_end_time = time.time()
total_training_time = total_end_time - total_start_time
print(f'Total Training Time: {total_training_time:.2f} seconds')

# Model Evaluation
model.eval()
correct = 0
total = 0
all_predicted = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = correct / total
print(f'Final Test Accuracy: {accuracy * 100:.2f}%')

# Calculate and print F1 score, precision, and recall
f1 = f1_score(all_labels, all_predicted, average='weighted')
precision = precision_score(all_labels, all_predicted, average='weighted')
recall = recall_score(all_labels, all_predicted, average='weighted')

print(f'Final F1 Score: {f1:.4f}')
print(f'Final Precision: {precision:.4f}')
print(f'Final Recall: {recall:.4f}')

# Calculate and print confusion matrix
cm = confusion_matrix(all_labels, all_predicted)
print('Final Confusion Matrix:')
print(cm)

Epoch 1/300, Loss: 1.8129211501087374, Training Time: 20.64 seconds
Epoch 11/300, Loss: 0.701896849343234, Training Time: 20.64 seconds
Epoch 21/300, Loss: 0.2185732681504296, Training Time: 20.70 seconds
Epoch 31/300, Loss: 0.033309594931704996, Training Time: 20.71 seconds
Epoch 41/300, Loss: 0.011184577406753245, Training Time: 20.68 seconds
Epoch 51/300, Loss: 0.008552617130829665, Training Time: 20.59 seconds
Epoch 61/300, Loss: 0.00843371897946288, Training Time: 20.62 seconds
Epoch 71/300, Loss: 0.0033418990282640766, Training Time: 20.73 seconds
Epoch 81/300, Loss: 0.0034298691896351334, Training Time: 20.61 seconds
Epoch 91/300, Loss: 0.003717368664292921, Training Time: 20.54 seconds
Epoch 101/300, Loss: 0.0023898073341673634, Training Time: 20.58 seconds
Epoch 111/300, Loss: 0.0054902860824012165, Training Time: 20.65 seconds
Epoch 121/300, Loss: 0.0025110696265457587, Training Time: 20.97 seconds
Epoch 131/300, Loss: 0.001742886721666115, Training Time: 20.70 seconds
Epoch 